In [16]:
pip install datasets tensorflow==2.16.1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
import datasets
import tensorflow as tf

In [3]:
ds = datasets.load_from_disk('../datasets/WikiArt_test')

In [18]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [19]:
tf.test.is_gpu_available()

2024-06-12 14:17:35.955227: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


False

In [4]:
model = timm.create_model('beit_large_patch16_512.in22k_ft_in22k_in1k', pretrained=True, num_classes=0)
model.eval() # turn on evaluation mode

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Beit(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-23): 24 x Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    

In [5]:
# save preprocessing information from the pretrained model
data_config = timm.data.resolve_model_data_config(model)

# use this information to transform the data
transforms = timm.data.create_transform(**data_config, is_training=False)

In [6]:
transforms

Compose(
    Resize(size=512, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(512, 512))
    ToTensor()
    Normalize(mean=tensor([0.5000, 0.5000, 0.5000]), std=tensor([0.5000, 0.5000, 0.5000]))
)

In [9]:
img = ds[0]['image']
transformed_image = transforms(img).unsqueeze(0)

In [11]:
type(transformed_image)

torch.Tensor

In [12]:
features = model(transformed_image)

In [15]:
len(features[0])

1024

In [ ]:
features = model(transforms(img).unsqueeze(0))

transformed_image = transforms(img).unsqueeze(0)